In [2]:
library(ggplot2)
library(reshape2)
library(tensorflow)
library(tfestimators)

set.seed(123)

## Generate Data
n = 200
mu1 = 10 ; mu2 = -3;mu3 = -12 ; mu4 = 5;
sig = 10
Var1 <- c(rnorm(n/2, mu1, sig), rnorm(n/2, mu2, sig))
Var2 <- c(rnorm(n/2, mu3, sig), rnorm(n/2, mu4, sig))
x <- matrix(c(Var1, Var2), nrow = n, ncol = 2)
outcome <- c(rep(1, n/2), rep(0, n/2))
main.data = data.frame(x,outcome)


In [ ]:

response <- function() "outcome"
features <- function() setdiff(names(main.data), response())



## Create Train and Test Data sets
partitions <- modelr::resample_partition(main.data, c(test = 0.2, train = 0.8))
md_train <- as.data.frame(partitions$train)
md_test  <- as.data.frame(partitions$test)


# construct feature columns
feature_columns <- feature_columns(
  column_numeric(features())
)

# construct a linear classifier
classifier1 <- linear_classifier(
  feature_columns = feature_columns,
  n_classes = 2
)
# construct a dnn classifier
classifier2 <- dnn_classifier(
  feature_columns = feature_columns,
  hidden_units = c(10, 20, 10),
  n_classes = 2
)


# construct input function 
md_input_fn <- function(data) {
  input_fn(data, features = features(), response = response())
}


# train classifier with training dataset
train(classifier1, input_fn = md_input_fn(md_train))
train(classifier2, input_fn = md_input_fn(md_train))



# valuate with test dataset
predictions1 <- predict(classifier1, input_fn = iris_input_fn(md_test))
evaluation1 <- evaluate(classifier1, input_fn = iris_input_fn(md_test))
classes1 <- vapply(1:nrow(predictions1), function(i) {
  predictions1$classes[[i]]
}, character(1))

predictions2 <- predict(classifier2, input_fn = iris_input_fn(md_test))
evaluation2 <- evaluate(classifier2, input_fn = iris_input_fn(md_test))
classes2 <- vapply(1:nrow(predictions2), function(i) {
  predictions2$classes[[i]]
}, character(1))

(error1 = mean(md_test$outcome!=as.numeric(classes1)))
(error2 = mean(md_test$outcome!=as.numeric(classes2)))
